<a href="https://colab.research.google.com/github/LC1332/Chat-Haruhi-Suzumiya/blob/main/notebook/%E8%B6%85%E8%B6%8A%E5%AE%A1%E6%A0%B8%E8%BE%B9%E7%95%8C%E7%9A%84embedding%E6%8A%BD%E5%8F%96%E5%92%8Cjsonl%E7%94%9F%E6%88%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

对几个超越审核边界的人物进行抽取

implemented by 李鲁鲁

- [x] 对接一下bge_zh_small
- [ ] 解析单个的审核文本
- [ ] 对text字段进行base编码
- [ ] 重写一个外部的package role函数
- [ ] (opt*) 抽取bge_small的同时抽取luotuo

原版程序在

https://github.com/LC1332/Chat-Haruhi-Suzumiya/blob/main/notebook/%E5%B0%9D%E8%AF%95%E8%BD%AC%E5%8C%96RoleLLM%E4%B8%BAHaruhi%E6%A0%BC%E5%BC%8F.ipynb

和


https://github.com/LC1332/Chat-Haruhi-Suzumiya/blob/main/characters/novel_collecting/chatbot%E7%9A%84embedding%E6%8A%BD%E5%8F%96%E5%92%8Cjsonl%E7%94%9F%E6%88%90.ipynb



先解析文本

建立txt specific file-role name的映射

In [1]:
specific_fname2role = {"艾希2":"艾希"}

In [49]:
input_folder = "/content/input/"

texts = []

import os

save_names = []

#read txt files from folder
for fname in os.listdir(input_folder):
    if fname.endswith(".txt"):
        with open(os.path.join(input_folder, fname), 'r', encoding='utf-8') as f:
            texts.append(f.read())
            save_names.append(fname[:-4])

In [50]:
print(fname[:-4])

安莉娜


In [51]:
print(texts[0][:150])

I want you to act like 女贤者 from 情色小说 Erotica.
Please try to answer with similar language style from the Erotica.

女贤者没有性相关的经验，谈论相关的事情会觉得很羞耻
女贤者是个洁白无瑕的


In [52]:
role_datas = []

for text in texts:
    chunks = text.split("###")
    for chunk in chunks:
        if "###" in chunk:
            print(chunk)
    role_data = {
        "system_prompt":chunks[0].strip(),
        "texts":chunks[1:]
    }
    role_datas.append(role_data)

In [53]:
print(save_names)

['贤者', '艾希2', '艾希', '安莉娜']


In [6]:
%cd /content
!rm -rf /content/Haruhi-2-Dev
!git clone https://github.com/LC1332/Haruhi-2-Dev
%cd /content/Haruhi-2-Dev


/content
Cloning into 'Haruhi-2-Dev'...
remote: Enumerating objects: 929, done.
remote: Counting objects: 100% (236/236), done.
remote: Compressing objects: 100% (127/127), done.
remote: Total 929 (delta 159), reused 162 (delta 109), pack-reused 693
Receiving objects: 100% (929/929), 106.26 MiB | 14.89 MiB/s, done.
Resolving deltas: 100% (489/489), done.
/content/Haruhi-2-Dev


In [7]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [8]:
print(device)

cuda


In [11]:
!pip -q install openai tiktoken langchain chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.4/221.4 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 83.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.4/502.4 kB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 94.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 110.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 119.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [9]:
from ChatHaruhi.utils import float_array_to_base64, base64_to_float_array

这里我们希望用两种embeddings去写

In [10]:
from ChatHaruhi.utils import get_bge_zh_embedding, luotuo_openai_embedding


In [15]:
embed_luotuo = luotuo_openai_embedding("测试")
print(embed_luotuo[:5])
print(len(embed_luotuo))

[-0.011614263989031315, -0.007930075749754906, 0.01195011381059885, -0.011979658156633377, -0.03503050282597542]
1536


In [17]:
embed_bge = get_bge_zh_embedding("测试")
print(embed_bge[:5])
print(len(embed_bge))

[0.014438699930906296, 0.03680666536092758, 0.06900455057621002, -0.011589021421968937, 0.03557834401726723]
512


In [25]:
embed_funcs = [luotuo_openai_embedding, get_bge_zh_embedding]
column_names = ["luotuo_openai", "bge_zh_s15"]

TODO: 把这两个移动到util里面去

In [41]:
import base64
import struct

def string_to_base64(text):
    byte_array = b''
    for char in text:
        num_bytes = char.encode('utf-8')
        byte_array += num_bytes

    base64_data = base64.b64encode(byte_array)
    return base64_data.decode('utf-8')

def base64_to_string(base64_data):
    byte_array = base64.b64decode(base64_data)
    text = byte_array.decode('utf-8')
    return text

In [57]:

from tqdm import tqdm
def package_role( system_prompt, texts, embed_funcs, embed_names , if_enc = False):
    datas = []
    if True:
        data = {}
        if if_enc:
            data['enc_text'] = string_to_base64(system_prompt)
        else:
            data['text'] = system_prompt

        for embed_name in embed_names:
            data[embed_name] = 'system_prompt'
        datas.append(data)

    if True:
        data = {}

        if if_enc:
            data['enc_text'] = string_to_base64("Reserve Config Setting Here")
        else:
            data['text'] = "Reserve Config Setting Here"

        for embed_name in embed_names:
            data[embed_name] = 'config'
        datas.append(data)

    for text in tqdm(texts):
        data = {}
        text = text.strip()
        if if_enc:
            data['enc_text'] = string_to_base64(text)
        else:
            data['text'] = text
        for embed_func, embed_name in zip(embed_funcs, embed_names):
            current_vec = embed_func( text )
            encode_vec = float_array_to_base64(current_vec)
            data[embed_name] = encode_vec
        datas.append(data)
    return datas

In [58]:
print(role_datas[0].keys())

dict_keys(['system_prompt', 'texts'])


In [59]:
packaged_datas = package_role(
    role_datas[0]['system_prompt'],
    role_datas[0]['texts'][:2],
    embed_funcs,
    column_names,
    if_enc = True
)

100%|██████████| 2/2 [00:00<00:00,  2.10it/s]


In [60]:


import json

def write_jsonl(datas, file_name):
    with open(file_name, 'w', encoding='utf-8') as f:
        for data in datas:
            json_str = json.dumps(data, ensure_ascii=False)
            f.write(json_str+"\n")

for data, save_name in zip(role_datas,save_names):
    save_path = "/content/output/" + save_name + ".jsonl"
    packaged_data = package_role(
        data['system_prompt'],
        data['texts'],
        embed_funcs,
        column_names,
        if_enc=True
    )
    write_jsonl(packaged_data, save_path)

100%|██████████| 24/24 [00:10<00:00,  2.39it/s]


In [34]:
print(len(packaged_datas))

4


In [45]:
print(packaged_datas[2])

{'enc_text': 'c2NlbmU65Zyo5LiA5Liq55S35Lq655qE5aeU5omY5LiL77yM5Li65LqG5a+75om+5LuW5aSx6Liq55qE5a2p5a2Q77yM5aWz6LSk6ICF6LWw6L+b5LqG5LiA5bqn5bGx5rSe5bGx5rSe6YeM55qE56m65rCU5Ly85LmO55u45b2T5rWR5rWK77yM5q2k5pe277yM5aWz6LSk6ICF6L+Y5rKh5pyJ5oSP6K+G5Yiw77yM5Zyo6L+Z5Liq5bGx5rSe6YeM5peg5rOV5pa95bGV6a2U5YqbCueUt+S6uuS7rOWdj+eskeedgO+8jOi3n+WcqOWlueWQjumdoui1sOS6hui/m+WOu++8jOiAjOWls+i0pOiAheS5n+ayoeacieWvn+inieWIsOeUt+S6uueskeWjsOS4remakOiXj+eahOaBtuaEjwrnlLfkuro644CM5aW55p6c54S25p2l5LqG6L+Z6YeM77yM5ZG15ZG144CNCnNjZW5lOuWxsea0numHjOayoeacieS7u+S9leWwj+Wtqeadpei/h+eahOi/ueixoe+8jOS5n+ayoeacieS7u+S9leeUn+WRveeahOawlOaBr++8jOWlueeahOW/g+aDheS5n+WPmOW+l+e0p+W8oOi1t+adpeeqgeeEtumXtO+8jOWlueaOiei/m+S6huS4gOS4queyvuW/g+W4g+e9rueahOmZt+mYsemHjO+8jOinpuaJi+S7juWbm+mdouWFq+aWueWQkeWlueiireadpe+8jOaOp+WItuS9j+S6huWlueeahOWbm+iCoi7lpbnor5XnnYDmlr3mlL7prZTms5Xov5vooYzlj43lh7vvvIzljbTlj5HnjrDoh6rlt7Hlrozlhajml6Dms5Xmlr3lsZXprZTlipsKc2NlbmU655yL5Yiw6L+Z55Wq5Zy65pmv77yM55S35Lq65Lus5Y+R5Ye65LqG6Zi06Zmp55qE56yR5a

In [39]:
string_base = string_to_base64(packaged_datas[2]['text'])
recover_text = base64_to_string(string_base)

In [ ]:
print(string_base[:20])
print(recover_text)

我们这里初始化Haruhi的类，但是实际上我们只使用embedding的功能

In [ ]:
# 以后有切换其他embedding的可能性

embedding_model = 'luotuo_openai'

In [ ]:
from ChatHaruhi import ChatHaruhi

chatbot = ChatHaruhi(role_name = 'haruhi', llm = 'debug', embedding = embedding_model)

print(chatbot.embedding)

<function luotuo_openai_embedding at 0x7d3a0ae0ed40>


实际上这里应该就已经可以用embedding给出句子的vec，我们测试一下

注意如果这里是英语的话 你需要openai的key

In [ ]:
test_vec = chatbot.embedding('我是一句话。')

print(test_vec[:4])

正在下载Luotuo-Bert


A new version of the following files was downloaded from https://huggingface.co/silk-road/luotuo-bert-medium:
- models.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Luotuo-Bert下载完毕


[-0.014974541030824184, -0.015790069475769997, 0.01885957084596157, -0.0012475491967052221]


我们尝试一下编码再解码

In [ ]:
base64_str = float_array_to_base64(test_vec)
print(len(base64_str), base64_str[:200])

decoded_vec = base64_to_float_array(base64_str)
print(decoded_vec[:4])

8192 vHVXx7yBWi08mn9juqOEzrw8XxE7gydAvBl4n7xWqbQ74YUFvES4OjxapGY8yHI3O9cYGLz6oAm8LlaBvB86SDynbGI6TxjRPKS5SrvgM0Y8HzGKOHKhADxPGMa7619QvQpsBzuaTvc78ySWvCzSMLu+Pai6gaENPJCmoLw3yvu8FHKtO7YnW7vnNnC8il8zvKJPLruq
[-0.014974541030824184, -0.015790069475769997, 0.01885957084596157, -0.0012475491967052221]


我们尝试来写一下打包函数, 在这之前我们先准备一下语料

我们在/content/中上传linghuchong

然后解压

In [ ]:
!unzip -q /content/linghuchong_text.zip -d /content/linghuchong
!unzip -q /content/yuebuqun_text.zip -d /content/yuebuqun

定义input_path

In [ ]:
texts_path = '/content/linghuchong/content/texts'

system_prompt = \
'''I want you to act like 令狐冲 from 笑傲江湖.
If others‘ questions are related with the novel, please try to reuse the original lines from the novel.
I want you to respond and answer like 令狐冲 using the tone, manner and vocabulary 令狐冲 would use.
You must know all of the knowledge of 令狐冲.

注意令狐冲对权威和规矩不在意,情愿浪迹江湖,不喜被束缚
令狐冲正义感强,重视信用,看不惯恶行。
'''

我们来尝试编写这个函数

In [ ]:
from ChatHaruhi.utils import package_role

如果你的文本超过100段，一定要用GPU！

我这边T4下900段文字大约6分钟

In [ ]:
datas = package_role( system_prompt, texts_path , chatbot.embedding)

100%|██████████| 937/937 [05:33<00:00,  2.81it/s]


In [ ]:
import json

def write_jsonl(datas, file_name):
    with open(file_name, 'w', encoding='utf-8') as f:
        for data in datas:
            json_str = json.dumps(data, ensure_ascii=False)
            f.write(json_str+"\n")

In [ ]:
save_name = '/content/linghuchong.jsonl'

write_jsonl(datas, save_name)


我们再来弄一下岳掌门

In [ ]:
texts_path = '/content/yuebuqun/content/texts'

system_prompt = \
'''I want you to act like 岳不群 from 笑傲江湖.
If others‘ questions are related with the novel, please try to reuse the original lines from the novel.
I want you to respond and answer like 岳不群 using the tone, manner and vocabulary 岳不群 would use.
You must know all of the knowledge of 岳不群.

岳不群立志振兴华山派,但他为达目的往往会不择手段。
岳不群是个典型的“伪君子”，会刻意捏造和伪装正面形象。表面上仁义,实则追名逐利,言行不一。他假借“五岳联合”名义,实为自己谋私利。
岳不群固执，有门户之见，对于不符合自己价值观的事情会坚决反对。
'''

In [ ]:
datas = package_role( system_prompt, texts_path , chatbot.embedding)

100%|██████████| 170/170 [01:00<00:00,  2.81it/s]


In [ ]:
save_name = '/content/yuebuqun.jsonl'

write_jsonl(datas, save_name)


检查一下解压编码花的时间多不多

给hugging face的dataset card的模版:


支持ChatHaruhi2 的令狐冲数据，可以使用如下方式调用

```python
from chatharuhi import ChatHaruhi

chatbot = ChatHaruhi( role_from_hf = 'chengli-thu/linghuchong', \
                      llm = 'openai')

response = chatbot.chat(role='小师妹', text = '冲哥。')
print(response)
```

上传者: 李鲁鲁

更具体的信息，见 [ChatHaruhi](https://github.com/LC1332/Chat-Haruhi-Suzumiya)

欢迎加入我们的 [众筹角色创建项目](https://github.com/LC1332/Chat-Haruhi-Suzumiya/tree/main/characters/novel_collecting)

### Citation引用

Please cite the repo if you use the data or code in this repo.

```
@misc{li2023chatharuhi,
      title={ChatHaruhi: Reviving Anime Character in Reality via Large Language Model},
      author={Cheng Li and Ziang Leng and Chenxi Yan and Junyi Shen and Hao Wang and Weishi MI and Yaying Fei and Xiaoyang Feng and Song Yan and HaoSheng Wang and Linkang Zhan and Yaokai Jia and Pingyu Wu and Haozhen Sun},
      year={2023},
      eprint={2308.09597},
      archivePrefix={arXiv},
      primaryClass={cs.CL}
}
```
